## Lib

In [ ]:
!pip install openai==1.2.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.9/219.9 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.0 MB/s eta 0:00:00


In [ ]:
import os
import pandas as pd
import numpy as np
import json
import openai
from openai import OpenAI
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
from collections import Counter
import ast

In [ ]:
client = OpenAI(
  api_key="Your OpenAI API here"
)
def get_completion(prompt, model = "gpt-3.5-turbo", temperature =0):
  messages = [{"role":"user", "content": prompt}]
  response = client.chat.completions.create(
      model = model,
      messages = messages,
      temperature =temperature # degree of expiration (randomness) (0 same result - 1 creative)
  )
  return response.choices[0].message.content

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Technical Issues

In [ ]:
def tech(data):
  duration = data['Utterance end time (milliseconds)'].iloc[-1]
  start = 0
  end = 1
  l = []
  idx = []
  num = data['Utterance end time (milliseconds)'].iloc[-1]/60000/20
  for i in range(len(data)):
    if data['Utterance end time (milliseconds)'].iloc[i] > (end*60000*num):
        d = data[['Speaker', 'Utterance']].iloc[start:i+1].to_json()
        prompt = f"""I provided you a chunk of transcript between a student and a tutor in a online one-on-one tutoring session{d}.
        You task is to identidy that whether the tutor is experiencing a technnological issue with the plateform.
        Here are some examples of technique issue: donot know how to use a tool, cannot hear each other, cannot see each other, cannot upload a file, cannot share the screen, or cannot see the screen.
        Output only 1 for yes or 0 for no."""
        r = get_completion(prompt)
        l.append(r[0])
        idx.append([start,i+1])
        start = i+1
        end = end+1
  d = data[['Speaker', 'Utterance']].iloc[start:len(data)-1].to_json()
  prompt = f"""I provided you a chunk of transcript between a student and a tutor in a online one-on-one tutoring session{d}.
        You task is to identidy that whether the tutor is experiencing a technical issure.
        Here are some examples of technique issue: donot know how to use a tool, cannot hear each other, cannot see each other, cannot upload a file, cannot share the screen, or cannot see the screen.
        Output only 1 for yes or 0 for no."""
  r = get_completion(prompt)
  l.append(r[0])
  idx.append([start,i+1])

  total_tech_score = sum(list(map(int, l)))/len(l)
  return(l, idx, total_tech_score)

## Communication

In [ ]:
def comm(data):
  start = 0
  end = 1
  l = []
  idx = []
  num = data['Utterance end time (milliseconds)'].iloc[-1]/60000/20
  for i in range(len(data)):
    if data['Utterance end time (milliseconds)'].iloc[i] > (end*60000*num):
      c = 0
      for j in data['DA'].iloc[start:i+1]:
        if j in ['Extra Domain Other','Extra Domain Question', 'Extra Domain Answer', 'Greeting']:
          c = 1
          break
      l.append(c)
      idx.append([start,i+1])
      start = i+1
      end = end+1
  c=0
  for j in data['DA'].iloc[start:len(data)-1]:
      if j in ['Extra Domain Other','Extra Domain Question', 'Extra Domain Answer', 'Greeting']:
        c = 1
        break
  l.append(c)
  idx.append([start,i+1])

  p = Counter(list(d['Speaker']))['student']/len(list(d['Speaker']))

  total_comm_score=(sum(list(map(int, l)))/len(l))/2

  step = round(data['Utterance end time (milliseconds)'].iloc[-1]/60000/20)
  time = [f"{i:02}" for i in range(step, step*20+1, step)]

  return (l,idx, total_comm_score, time)

## Instruction

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

tokenizer_1 = AutoTokenizer.from_pretrained("s-nlp/xlmr_formality_classifier")
model_1 = AutoModelForSequenceClassification.from_pretrained("s-nlp/xlmr_formality_classifier")
config_1 = AutoConfig.from_pretrained("s-nlp/xlmr_formality_classifier")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
def instru(data):
  l = []
  for i in range(len(data)):
    text = data.iloc[i]['Utterance']
    encoded_input = tokenizer_1(text, return_tensors='pt')
    output = model_1(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    # Print labels and scores
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    l.append(config_1.id2label[ranking[0]])
  data['formality'] = l
  start = 0
  end = 1
  l = []
  idx = []
  lb = []
  num = data['Utterance end time (milliseconds)'].iloc[-1]/60000/20
  for i in range(len(data)):
    if data['Utterance end time (milliseconds)'].iloc[i] > (end*60000*num):
      c = 0
      b = 0
      for j in range(start,i+1):
        if (data['DA'].iloc[j]=='Explanation') & (data['Speaker'].iloc[j]=='student'):
          c += 1
      for j in data['formality'].iloc[start:i+1]:
        if j == 'informal':
          b += 1
      l.append(c)
      lb.append(b)
      idx.append([start,i+1])
      start = i+1
      end = end+1
  c=0
  b= 0
  for j in range(start,len(data)-1):
      if (data['DA'].iloc[j]=='Explanation') & (data['Speaker'].iloc[j]=='student'):
        c += 1
  for j in data['formality'].iloc[start:i+1]:
      if j == 'informal':
        b += 1
  lb.append(b)
  l.append(c)
  idx.append([start,i+1])

  total_instru_score = sum(list(map(int, l)))/len(data) - sum(list(map(int, lb)))/len(data)
  return (lb,idx, total_instru_score)

## Social emotion

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)

model = AutoModelForSequenceClassification.from_pretrained(MODEL)


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def socialEmo(data):
  l = []
  for i in range(len(data)):
    text = data.iloc[i]['Utterance']
    text = preprocess(text)
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    # Print labels and scores
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    l.append(config.id2label[ranking[0]])
  data['tone'] = l
  stu_neg = sum((data['tone']=='negative') & (data['Speaker']=='student'))/ sum( (data['Speaker']=='student'))
  tutor_neg = sum((data['tone']=='negative') & (data['Speaker']=='student'))/ sum( (data['Speaker']=='tutor'))

  start = 0
  end = 1
  l = []
  idx = []
  num = data['Utterance end time (milliseconds)'].iloc[-1]/60000/20
  for i in range(len(data)):
    if data['Utterance end time (milliseconds)'].iloc[i] > (end*60000*num):
      c = 0
      for j in data['tone'].iloc[start:i+1]:
        if j == 'negative':
          c += 1
      l.append(c/len(data))
      idx.append([start,i+1])
      start = i+1
      end = end+1
    c=0
  for j in data['tone'].iloc[start:len(data)-1]:
      if j == 'negative':
        c += 1
  l.append(c/len(data))
  idx.append([start,i+1])

  return (stu_neg,tutor_neg, l)

## Feedback


In [ ]:
def feedback(data):
  start = 0
  end = 1
  l = []
  lb = []
  idx = []
  totalcount = 0
  num = data['Utterance end time (milliseconds)'].iloc[-1]/60000/20
  for i in range(len(data)):
    if data['Utterance end time (milliseconds)'].iloc[i] > (end*60000*num):
      c = 0
      b = 0
      for j in data['DA'].iloc[start:i+1]:
        if j in ['Positive Feedback','Negative Feedback']:
          totalcount +=1
          if i+4 < len(data):
            temp = data.iloc[i:i+4]['DA']
            if ('Explanation' in list(temp)):
              c += 1
            else:
              b += 1
      l.append(c)
      lb.append(b/len(data))
      idx.append([start,i+1])
      start = i+1
      end = end+1
  c = 0
  b = 0
  for j in data['DA'].iloc[start:len(data)-1]:
      if j in ['Positive Feedback','Negative Feedback']:
          totalcount+=1
          if i+4 < len(data):
            temp = data.iloc[i:i+4]['DA']
            if ('Explanation' in list(temp)):
              c += 1
            else:
              b +=1
  l.append(c)
  lb.append(b/len(data))
  idx.append([start,i+1])

  total_feedback_score =  sum(list(map(int, l)))/totalcount
  return (lb,idx, total_feedback_score)

# Running Evaluation 

In [ ]:
raw = pd.read_csv('/Data/500 Sample Supplementary Data (Laste edit_ Mar 21) - mar20 base tutoring session data.csv')

In [ ]:
dir = 'Directory to the DA labled transcripts'
n=os.listdir(dir)

In [ ]:
data = raw
total_tech_score = []
total_comm_score = []
total_instru_score = []
total_feedback_score = []
stu_neg = []
tutor_neg = []
duration = []

interval = []
t_score = []
c_score = []
i_score = []
f_score = []
social_emo = []

time = []

r = pd.DataFrame()

for i in n:
  if i[:-4] in list(raw['session_uid']):
    r = pd.concat([r, raw[raw['session_uid'] == i[:-4]]])

    d = pd.read_csv(dir+i)

    tech_score, tech_interval, s = tech(d)
    total_tech_score.append(s)
    t_score.append(tech_score)

    comm_score, comm_interval, s, t = comm(d)
    total_comm_score.append(s)
    c_score.append(comm_score)

    instru_score, instru_interval, s = instru(d)
    total_instru_score.append(s)
    i_score.append(instru_score)
    print(instru_score)

    feedback_score, feedback_interval, s = feedback(d)
    total_feedback_score.append(s)
    f_score.append(feedback_score)

    stu, tu, s = socialEmo(d)
    stu_neg.append(stu)
    tutor_neg.append(tu)
    social_emo.append(s)

    interval.append(feedback_interval)

    du = raw[raw['session_uid'] == i[:-4]]
    duration.append(list(d['Utterance end time (milliseconds)'].iloc[-1]/60000/du['Sessions duration (min)'])[0])

    time.append(t)

r['total_tech_score'] = total_tech_score
r['total_comm_score'] = total_comm_score
r['total_instru_score'] = total_instru_score
r['total_feedback_score'] = total_feedback_score
r['stu_neg'] = stu_neg
r['tutor_neg'] = tutor_neg
r['duration'] = duration

r['interval'] = interval
r['t_score'] = t_score
r['social_emo'] = social_emo
r['c_score'] = c_score
r['i_score'] = i_score
r['f_score'] = f_score

r['time'] = time

# Calculate Scores 0-10

In [ ]:
r['total_tech'] = round(1- r['total_tech_score'],2)*10

In [ ]:
r['total_comm'] = round(1- r['total_comm_score'],2)*10

In [ ]:
r['socialemo'] = 5*r['stu_neg']+5*r['tutor_neg']
m = np.mean(round(1-r['socialemo']*100,2))
std = np.std(round(1-r['socialemo']*100,2))
y = -(round(1-r['socialemo']*100,2)-m)/std/np.sqrt(2)
r['total_socialemo'] = round(scipy.special.erfc(y, out=None)*5,1)

In [ ]:
m = np.mean(round(r['total_instru_score']*100,2))
std = np.std(round(r['total_instru_score']*100,2))
y = -(round(r['total_instru_score']*100,2)-m)/std/np.sqrt(2)
r['total_instru'] = round(scipy.special.erfc(y, out=None)*5,1)

In [ ]:
r['total_feedback']=round(r['total_feedback_score']*10,1)

In [ ]:
r['total'] = (r['total_tech'] +r['total_comm']+r['total_socialemo']+r['total_instru']+r['total_feedback'])/5

In [ ]:
r.to_csv('Directory to save the output file')

# Output Json Files for Frontend

## first page

In [ ]:
def jsondict(s):
  risklist = []
  score_label = {"text" : "Low Rubric Rating", "color": "aquamarine"}
  tech_label = {"text" : "Technical Issues","color": "blue"}
  duration_label =  {"text" : "Short Sessions","color": "yellow"}

  dr = (s['duration'] * s['Sessions duration (min)']).astype('int64')
  d = s['Sessions duration (min)']

  if s['total_tech_score']>=0.3:
    risklist.append(tech_label)
  if s['total'] < 6.5:
    risklist.append(score_label)
  if s['duration']<0.8:
    risklist.append(duration_label)
  dict =   {"sessionId": s['session_uid'],
            "tutor": s["Tutor Name"],
            "subject": s["subject"],
            "date": s["tutoring session occurred date"],
            "duration": str(dr) + '/' + str(d),
            "rubircRating": s['total'],
            "sessionDuration": s['duration'],
            "viewUrl": "https://google.com",
            "riskAreas": risklist}
  return dict

In [ ]:
result = []
for i in range(len(r)):
  s = r.iloc[i]
  result.append(jsondict(s))
json_object = json.dumps(result, indent=2)

In [ ]:
with open("/Directory for Json Output/list_first_page.json", "w") as outfile:
    outfile.write(json_object)

## second page

In [ ]:
for i in range(len(r)):
  s = r.iloc[i]
  a = ast.literal_eval(s['time'])
  a.insert(0, '00')
  dict = {
    "communicationLevelInfo": {
      "level": round(r['total_comm'].iloc[i],1),
      "percent": round(sum((r['total_comm']<=r['total_comm'].iloc[i]))/len(r['total_comm'])*100,1),
      "remarks": [
        "· This score measures communication in the session in the session.",
        "· Students are expected to actively participate in course content.",
        "· The score is measured by the turns and extra domain discussion."
      ]
    },
    "instructionDeliveryInfo": {
      "level": round(r['total_instru'].iloc[i],1),
      "percent": round(sum((r['total_instru']<=r['total_instru'].iloc[i]))/len(r['total_instru'])*100,1),
      "remarks": [
        "· This score measures tutor's instruction techniques in the session.",
        "· Turors are expected to check student understanding by ask them to explain.",
        "· The score is measured by the frequency."
      ]
    },
    "techToolUsageInfo": {
      "level": round(r['total_tech'].iloc[i],1),
      "percent": round(sum((r['total_tech']<=r['total_tech'].iloc[i]))/len(r['total_tech'])*100,1),
      "remarks": [
        "· This score measures technical difficulties in the session.",
        "· Tutors are expected to spend less time on technical difficulties.",
        "· The score is measured by the time."
      ]
    },
    "socioEmotionalTeachingInfo": {
      "level": round(r['total_socialemo'].iloc[i],1),
      "percent": round(sum((r['total_socialemo']<=r['total_socialemo'].iloc[i]))/len(r['total_socialemo'])*100,1),
      "remarks": [
        "· This score measures technical difficulty in the session.",
        "· Tutors and students are expected to use more positive tones.",
        "· The score is measured by the proportion of negative tones."
      ]
    },
    "feedbackInfo": {
      "level": round(r['total_feedback'].iloc[i],1),
      "percent": round(sum((r['total_feedback']<=r['total_feedback'].iloc[i]))/len(r['total_feedback'])*100,1),
      "remarks": [
        "· This score measures tutor's feedback quality in the session.",
        "· Tutors are expected to provide detailed feedback with explainations.",
        "· The score is measured by the proportion of good feedback."
      ]
    },
    "timeline": {
      "timeText": a[:20],
      "communicationLevel": [int(n)*2 for n in ast.literal_eval(r['c_score'].iloc[i])][:20],
      "instructionDelivery": [max(round(n /( max(ast.literal_eval(r['i_score'].iloc[i]))/4)-1),0) for n in ast.literal_eval(r['i_score'].iloc[i])][:20],
      "techToolUsage": [int(n)*2 for n in ast.literal_eval(r['t_score'].iloc[i])][:20],
      "socioEmotionalTeaching": [max(round(n /( max(ast.literal_eval(r['social_emo'].iloc[i]))/4)-1),0) for n in ast.literal_eval(r['social_emo'].iloc[i])][:20],
      "feedback": [max(round(n /( max(ast.literal_eval(r['f_score'].iloc[i]))/4)-1),0) for n in ast.literal_eval(r['f_score'].iloc[i])][:20]
    }

  }
  json_object = json.dumps(dict, indent=2)
  with open("/Directory for Json Output/detail_"+ r['session_uid'].iloc[i] +".json", "w") as outfile:
      outfile.write(json_object)